# Chinese exports by year, country and commodity

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import glob

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: "%.2f" % x)

---

### Read processed trade data

In [4]:
df = pd.read_csv(
    "../data/processed/exports_xingjiang_all_countries.csv",
    dtype={
        "commodity_code": str,
        "commodity_chapter": str,
        "commodity_chapter_four": str,
    },
)

In [5]:
df.head()

,year,commodity_code,commodity_chapter,commodity_chapter_desc,commodity_chapter_four,commodity_chapter_four_desc,us_dollar,exported_to,exported_from
0,2019,02071419,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",259200,Kyrgyzstan,Xingjiang Uygur
1,2019,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",3584460,Kyrgyzstan,Xingjiang Uygur
2,2018,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",208000,Kyrgyzstan,Xingjiang Uygur
3,2017,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",106250,Kyrgyzstan,Xingjiang Uygur
4,2020,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",1113600,Kyrgyzstan,Xingjiang Uygur


---

### Exports to countries

In [6]:
xinjiang_exports = df.groupby(["exported_to"]).agg({"us_dollar": sum}).reset_index()

In [7]:
xinjiang_exports.sort_values("us_dollar", ascending=False).head(20)

In [27]:
xinjiang_exports.to_csv("xinjiang_exports_countries_2017_2021.csv", index=False)

---

### Most exported commodity types

In [8]:
df.head()

,year,commodity_code,commodity_chapter,commodity_chapter_desc,commodity_chapter_four,commodity_chapter_four_desc,us_dollar,exported_to,exported_from
0,2019,02071419,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",259200,Kyrgyzstan,Xingjiang Uygur
1,2019,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",3584460,Kyrgyzstan,Xingjiang Uygur
2,2018,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",208000,Kyrgyzstan,Xingjiang Uygur
3,2017,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",106250,Kyrgyzstan,Xingjiang Uygur
4,2020,02074500,02,Meat and edible meat offal,0207,"Meat and edible offal of poultry; of the poultry of heading no. 0105, (i.e. fowls of the species Gallus domesticus), fresh, chilled or frozen",1113600,Kyrgyzstan,Xingjiang Uygur


In [38]:
xinjiang_exports_types = (
    df.groupby(["commodity_chapter_desc"]).agg({"us_dollar": sum}).reset_index()
)

In [39]:
xinjiang_exports_types.sort_values("us_dollar", ascending=False).head(10)

,commodity_chapter_desc,us_dollar
6,Apparel and clothing accessories; not knitted or crocheted,12361523418
30,Footwear; gaiters and the like; parts of such articles,8530093335
5,Apparel and clothing accessories; knitted or crocheted,7145433906
62,Plastics and articles thereof,4450636812
8,"Articles of leather; saddlery and harness; travel goods, handbags and similar containers; articles of animal gut (other than silk-worm gut)",2424928662
58,Organic chemicals,2136200606
21,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",2125848899
64,"Preparations of vegetables, fruit, nuts or other parts of plants",2012005412
31,"Fruit and nuts, edible; peel of citrus fruit or melons",1510275242
79,"Textiles, made up articles; sets; worn clothing and worn textile articles; rags",1494637674


In [41]:
xinjiang_exports.to_csv("xinjiang_exports_commodities_2017_2021.csv", index=False)

### Exports by year

In [11]:
xinjiang_exports_types_year = (
    df.groupby(["year", "commodity_chapter_desc"]).agg({"us_dollar": sum}).reset_index()
)

In [12]:
xinjiang_exports_types_year_pivot = (
    xinjiang_exports_types_year.pivot_table(
        columns="year", index="commodity_chapter_desc", values="us_dollar"
    )
    .fillna(0)
    .reset_index()
)

In [13]:
xinjiang_exports_types_year_pivot.columns = (
    xinjiang_exports_types_year_pivot.columns.astype(str)
)

In [14]:
xinjiang_exports_types_year_pivot["2021"] = xinjiang_exports_types_year_pivot[
    "2021"
].round(0)

In [15]:
xinjiang_exports_types_year_pivot.head()

year,commodity_chapter_desc,2017,2018,2019,2020,2021
0,"Aircraft, spacecraft and parts thereof",0.00,0.00,0.00,0.00,58259.00
1,Albuminoidal substances; modified starches; glues; enzymes,9823866.00,11648087.00,11273195.00,7815793.00,8122353.00
2,Aluminium and articles thereof,64529277.00,116933622.00,0.00,0.00,112382667.00
3,Animal or vegetable fats and oils and their cleavage products; prepared animal fats; animal or vegetable waxes,126051.00,104069.00,82822.00,48143.00,56431.00
4,Animal originated products; not elsewhere specified or included,17981235.00,17393941.00,10039977.00,8030283.00,10654213.00


In [ ]:
xinjiang_exports_types_pivot.sort_values("2020", ascending=False).head(10)

In [16]:
xinjiang_exports_types_year_pivot["change"] = (
    xinjiang_exports_types_year_pivot["2021"]
    - xinjiang_exports_types_year_pivot["2020"]
) / xinjiang_exports_types_year_pivot["2020"]

In [17]:
xinjiang_exports_types_year_pivot.sort_values("change", ascending=False).head(10)

year,commodity_chapter_desc,2017,2018,2019,2020,2021,change
0,"Aircraft, spacecraft and parts thereof",0.00,0.00,0.00,0.00,58259.00,inf
26,"Feathers and down, prepared; and articles made of feather or of down; artificial flowers; articles of human hair",37609389.00,74858938.00,123881490.00,0.00,119162940.00,inf
32,"Furniture; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings; lamps and lighting fittings, n.e.c.; illuminated signs, illuminated name-plates and the like; prefabricated buildings",0.00,0.00,0.00,0.00,642189079.00,inf
34,Glass and glassware,98204331.00,88250593.00,110485038.00,0.00,105031306.00,inf
35,Headgear and parts thereof,116546428.00,61497466.00,64661144.00,0.00,73175915.00,inf
37,Iron and steel,301944321.00,281736935.00,301229891.00,0.00,365289725.00,inf
38,Iron or steel articles,350163622.00,466211827.00,132305002.00,0.00,429421796.00,inf
40,Lead and articles thereof,88885.00,332250.00,0.00,0.00,81470.00,inf
46,Metal; miscellaneous products of base metal,0.00,0.00,0.00,0.00,204170295.00,inf
50,Miscellaneous manufactured articles,0.00,0.00,0.00,0.00,166273014.00,inf


### Exports by year

In [18]:
agg_exports_by_year = df.groupby(["year"]).agg({"us_dollar": sum}).reset_index()

In [19]:
agg_exports_by_year.us_dollar.pct_change() * 100

0      NaN
1   -12.39
2     1.94
3   -69.33
4   435.32
Name: us_dollar, dtype: float64

In [20]:
agg_exports_by_year.head()

,year,us_dollar
0,2017,13436995662
1,2018,11771866361
2,2019,12000016602
3,2020,3680144438
4,2021,19700596110


In [42]:
agg_exports_by_year.to_csv("xinjiang_exports_agg_years_2017_2021.csv", index=False)

---

### Cotton exports

In [21]:
xinjiang_52_exports = (
    df[df["commodity_chapter"] == "52"]
    .groupby(["commodity_chapter_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [22]:
xinjiang_52_exports.sort_values("us_dollar", ascending=False).head(20)

,commodity_chapter_desc,exported_to,us_dollar
27,Cotton,Kyrgyzstan,99004238
25,Cotton,Kazakhstan,61101229
66,Cotton,Vietnam,29191946
40,Cotton,Pakistan,22047789
59,Cotton,Tajikistan,16899388
24,Cotton,Italy,6559469
65,Cotton,Uzbekistan,5733623
4,Cotton,Bangladesh,5026244
62,Cotton,United Arab Emirates,3830263
56,Cotton,Spain,2470699


### Tomatoes

In [23]:
xinjiang_0702_exports = (
    df[df["commodity_chapter_four"] == "0702"]
    .groupby(["commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [24]:
xinjiang_0702_exports.sort_values("us_dollar", ascending=False).head()

,commodity_chapter_four_desc,exported_to,us_dollar
0,Tomatoes; fresh or chilled,Kazakhstan,21672786
3,Tomatoes; fresh or chilled,Russian Federation,8842666
1,Tomatoes; fresh or chilled,Kyrgyzstan,3816516
4,Tomatoes; fresh or chilled,Uzbekistan,122927
2,Tomatoes; fresh or chilled,Mongolia,4124


### Polysilicon

In [25]:
xinjiang_280461_exports = (
    df[df["commodity_code"].str.contains("280461")]
    .groupby(["year", "commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [26]:
xinjiang_280461_exports

,year,commodity_chapter_four_desc,exported_to,us_dollar
0,2017,"Hydrogen, rare gases and other non-metals",Pakistan,5893160
1,2019,"Hydrogen, rare gases and other non-metals",Italy,4854
2,2020,"Hydrogen, rare gases and other non-metals",Belgium,494
3,2020,"Hydrogen, rare gases and other non-metals",Croatia,1
4,2020,"Hydrogen, rare gases and other non-metals",Reunion,2
